In [217]:
import pandas as pd
import numpy as np
from column_dictionary import questions
import warnings
import os
warnings.filterwarnings("ignore")

In [218]:
%run pandas-missing-extension.ipynb

Agregando archivo para procesar con pandas

In [219]:
df = pd.read_csv(
    "./data/base_saludmental.csv",
    sep=";"
)

Convirtiendo titulos en variables más legibles

In [220]:
new_columns = [questions[col_name] for col_name in df.columns]
df. columns = new_columns
df.head()

,timestamp,confianza_vecinos,frecuencia_habla_vecinos_mes_pasado,frecuencia_habla_vecinos_mes_normal,frecuencia_habla_amigos_familia_mes_pasado,frecuencia_habla_amigos_familia_mes_normal,cumplimiento_autocuidado_personal,cumplimiento_autocuidado_familiares,cumplimiento_autocuidado_vecinos,estrategia_comunicacion_clara,...,num_personas_hogar,num_personas_5_menos,num_personas_6_18,num_personas_19_64,num_personas_65_mas,oportunidad_expresar_sentimientos,frecuencia_actividad_fisica_mes,exposicion_violencia_hogar,diagnostico_salud_mental,continuidad_tratamiento_salud_mental
0,"44018,34521",La mayoría,Nada,Algunas veces al mes,Casi todos los días,Algunas veces a la semana,5 Totalmente,5 Totalmente,5 Totalmente,Sí,...,Cuatro,Una,Una,Dos,Ninguna,Sí,Ninguna vez,1.0,No,No aplica
1,"44018,34595",Algunos,Nada,Nada,Casi todos los días,Algunas veces al mes,5 Totalmente,4,3,No,...,Dos,Ninguna,Ninguna,Una,Ninguna,Sí,Menos de una vez a la semana,1.0,Sí,Sí
2,"44018,34611",Algunos,Algunas veces a la semana,Algunas veces a la semana,Casi todos los días,Casi todos los días,5 Totalmente,5 Totalmente,3,Sí,...,Tres,Ninguna,Ninguna,Tres,Ninguna,Sí,Más de una vez a la semana,1.0,No,No aplica
3,"44018,35482",La mayoría,Algunas veces a la semana,Algunas veces a la semana,Casi todos los días,Casi todos los días,5 Totalmente,4,4,Sí,...,Cuatro,Ninguna,Ninguna,Cuatro,Ninguna,Sí,Más de una vez a la semana,1.0,Sí,No
4,"44018,35899",La mayoría,Algunas veces al mes,Algunas veces al mes,Algunas veces a la semana,Algunas veces a la semana,4,4,4,Sí,...,Una (vivo solo),Ninguna,Ninguna,Una,Ninguna,Sí,Una vez a la semana,1.0,No,No aplica


In [221]:
df['confianza_vecinos'].drop_duplicates()

0      La mayoría
1         Algunos
8         Ninguno
9           Todos
836           NaN
Name: confianza_vecinos, dtype: object

In [222]:
dict_frecuency = {
    'Todos':4,
    'La mayoría':3,
    'Algunos':2,
    'Ninguno':1
}

Esta función me permite reemplazar los valores de una columna con un diccionario que traduzca los datos categoricos a numéricos

In [223]:
def replace_values(row,dict):
    if row is not np.nan:
        return dict[row]
    else:
        return np.nan

Esta función me permite aplicar la función anterior a cada valor del campo y reemplazar los valores nulos con la media de respuesta. Aplico la media debido a que los valores faltantes no superan un 6.20% salvo la pregunta final que solo se contesta dependiendo si la penultima es respondida con un sí.

In [224]:
def convert_colum_categorical_to_numeric(field_name,dict):
    df[field_name] = df[field_name].apply(lambda row: replace_values(row,dict))
    df[field_name] = df[field_name].fillna(int(df[field_name].mean()))

## Conversión de categorías a número
Cada una de las variables es analizada individualmente y se aplica un tipo de diccionario dependiendo de la pregunta para distribuir los pesos de la respuesta

In [225]:
field_name = 'confianza_vecinos'
convert_colum_categorical_to_numeric(field_name,dict_frecuency)
df[field_name].drop_duplicates().reset_index(drop=True)

0    3.0
1    2.0
2    1.0
3    4.0
Name: confianza_vecinos, dtype: float64

In [226]:
df['frecuencia_habla_vecinos_mes_pasado'].drop_duplicates().reset_index(drop=True)

0                         Nada
1    Algunas veces a la semana
2         Algunas veces al mes
3                      Una vez
4          Casi todos los días
5               No está seguro
6                          NaN
Name: frecuencia_habla_vecinos_mes_pasado, dtype: object

In [227]:
dict_frecuency2 = {
    'Casi todos los días':6,
    'Algunas veces a la semana':5,
    'Algunas veces al mes':4,
    'No está seguro':3,
    'Una vez':2,
    'Nada':1
}

In [228]:
field_name = 'frecuencia_habla_vecinos_mes_pasado'
convert_colum_categorical_to_numeric(field_name,dict_frecuency2)
df[field_name].drop_duplicates().reset_index(drop=True)

0    1.0
1    5.0
2    4.0
3    2.0
4    6.0
5    3.0
Name: frecuencia_habla_vecinos_mes_pasado, dtype: float64

In [229]:
field_name = 'frecuencia_habla_vecinos_mes_normal'
convert_colum_categorical_to_numeric(field_name,dict_frecuency2)
df[field_name].drop_duplicates().reset_index(drop=True)

0    4.0
1    1.0
2    5.0
3    2.0
4    6.0
5    3.0
Name: frecuencia_habla_vecinos_mes_normal, dtype: float64

In [230]:
field_name = 'frecuencia_habla_amigos_familia_mes_pasado'
convert_colum_categorical_to_numeric(field_name,dict_frecuency2)
df[field_name].drop_duplicates().reset_index(drop=True)

0    6.0
1    5.0
2    4.0
3    1.0
4    2.0
5    3.0
Name: frecuencia_habla_amigos_familia_mes_pasado, dtype: float64

In [231]:
field_name = 'frecuencia_habla_amigos_familia_mes_normal'
convert_colum_categorical_to_numeric(field_name,dict_frecuency2)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5.0
1    4.0
2    6.0
3    2.0
4    3.0
5    1.0
Name: frecuencia_habla_amigos_familia_mes_normal, dtype: float64

In [232]:
df['cumplimiento_autocuidado_personal'].drop_duplicates().reset_index(drop=True)

0    5 Totalmente
1               4
2               3
3          1 Nada
4               2
Name: cumplimiento_autocuidado_personal, dtype: object

In [233]:
dict_frecuency3 = {
        '5 Totalmente':5,
        '4':4,
        '3':3,
        '2':2,
        '1 Nada':1
    }

In [234]:
field_name = 'cumplimiento_autocuidado_personal'
convert_colum_categorical_to_numeric(field_name,dict_frecuency3)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5
1    4
2    3
3    1
4    2
Name: cumplimiento_autocuidado_personal, dtype: int64

In [235]:
field_name = 'cumplimiento_autocuidado_familiares'
convert_colum_categorical_to_numeric(field_name,dict_frecuency3)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5
1    4
2    3
3    1
4    2
Name: cumplimiento_autocuidado_familiares, dtype: int64

In [236]:
field_name = 'cumplimiento_autocuidado_vecinos'
convert_colum_categorical_to_numeric(field_name,dict_frecuency3)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5
1    3
2    4
3    2
4    1
Name: cumplimiento_autocuidado_vecinos, dtype: int64

In [237]:
df['estrategia_comunicacion_clara'].drop_duplicates().reset_index(drop=True)

0         Sí
1         No
2    No sabe
Name: estrategia_comunicacion_clara, dtype: object

In [238]:
dict_frecuency4 = {
    'Sí':2,
    'No':1,
    'No sabe':0,
    'No está seguro':0
}

In [239]:
field_name = 'estrategia_comunicacion_clara'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2
1    1
2    0
Name: estrategia_comunicacion_clara, dtype: int64

In [240]:
field_name = 'estrategia_comunicacion_suficiente'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2
1    1
2    0
Name: estrategia_comunicacion_suficiente, dtype: int64

In [241]:
field_name = 'estrategia_comunicacion_veridica'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2
1    0
2    1
Name: estrategia_comunicacion_veridica, dtype: int64

In [242]:
field_name = 'estrategia_comunicacion_oportuna'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2
1    1
2    0
Name: estrategia_comunicacion_oportuna, dtype: int64

In [243]:
field_name = 'estrategia_comunicacion_contradictoria'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    1
1    2
2    0
Name: estrategia_comunicacion_contradictoria, dtype: int64

In [244]:
field_name = 'sabe_actuar_sospechas_contagio'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2.0
1    1.0
Name: sabe_actuar_sospechas_contagio, dtype: float64

In [245]:
field_name = 'diagnostico_COVID'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    1.0
1    2.0
Name: diagnostico_COVID, dtype: float64

In [246]:
field_name = 'info_suficiente_COVID'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2.0
1    1.0
Name: info_suficiente_COVID, dtype: float64

In [247]:
df['info_riesgos_COVID_alcaldia'].drop_duplicates().reset_index(drop=True)

0       Precisa
1     Exagerada
2    Minimizada
3           NaN
Name: info_riesgos_COVID_alcaldia, dtype: object

In [248]:
dict_frecuency5 = {
    'Precisa':2,
    'Exagerada':1,
    'Minimizada':0
}

In [249]:
field_name = 'info_riesgos_COVID_alcaldia'
convert_colum_categorical_to_numeric(field_name,dict_frecuency5)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2.0
1    1.0
2    0.0
Name: info_riesgos_COVID_alcaldia, dtype: float64

In [250]:
df['ansiedad_cuarentena'].drop_duplicates().reset_index(drop=True)

0        Mucho más que antes
1      Un poco más que antes
2      Mucho menos que antes
3            Igual que antes
4    Un poco menos que antes
Name: ansiedad_cuarentena, dtype: object

In [251]:
dict_frecuency6 = {
    'Mucho más que antes':5,
    'Un poco más que antes':4,
    'Igual que antes':3,
    'Un poco menos que antes':2,
    'Mucho menos que antes':1,
    'No consumo':0
}

In [252]:
field_name = 'ansiedad_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5
1    4
2    1
3    3
4    2
Name: ansiedad_cuarentena, dtype: int64

In [253]:
field_name = 'depresion_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5
1    4
2    1
3    3
4    2
Name: depresion_cuarentena, dtype: int64

In [254]:
field_name = 'tranquilidad_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2
1    5
2    3
3    1
4    4
Name: tranquilidad_cuarentena, dtype: int64

In [255]:
field_name = 'cercania_convive_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5
1    4
2    3
3    1
4    2
Name: cercania_convive_cuarentena, dtype: int64

In [256]:
field_name = 'interes_placer_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2
1    5
2    3
3    4
4    1
Name: interes_placer_cuarentena, dtype: int64

In [257]:
field_name = 'problemas_dormir_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    4
1    1
2    3
3    2
4    5
Name: problemas_dormir_cuarentena, dtype: int64

In [258]:
field_name = 'desacuerdos_convive_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    3
1    4
2    2
3    5
4    1
Name: desacuerdos_convive_cuarentena, dtype: int64

In [259]:
field_name = 'necesidad_alcohol_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    0
1    2
2    4
3    3
4    1
5    5
Name: necesidad_alcohol_cuarentena, dtype: int64

In [260]:
field_name = 'necesidad_fumar_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    0
1    1
2    3
3    2
4    4
5    5
Name: necesidad_fumar_cuarentena, dtype: int64

In [261]:
field_name = 'necesidad_sustancias_psicoactivas_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency6)
df[field_name].drop_duplicates().reset_index(drop=True)

0    0
1    3
2    1
3    2
4    4
5    5
Name: necesidad_sustancias_psicoactivas_cuarentena, dtype: int64

In [262]:
df['apoyo_pareja_cuarentena'].drop_duplicates().reset_index(drop=True)

0    5 Todo el que necesito
1                    1 Nada
2                         4
3                       NaN
4                         3
5                         2
Name: apoyo_pareja_cuarentena, dtype: object

In [263]:
dict_frecuency7 = {
    '5 Todo el que necesito':5,
    '4':4,
    '3':3,
    '2':2,
    '1 Nada':1
}

In [264]:
field_name = 'apoyo_pareja_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency7)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5.0
1    1.0
2    4.0
3    3.0
4    2.0
Name: apoyo_pareja_cuarentena, dtype: float64

In [265]:
field_name = 'apoyo_familia_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency7)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5.0
1    2.0
2    4.0
3    3.0
4    1.0
Name: apoyo_familia_cuarentena, dtype: float64

In [266]:
field_name = 'apoyo_amigos_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency7)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5.0
1    3.0
2    4.0
3    2.0
4    1.0
Name: apoyo_amigos_cuarentena, dtype: float64

In [267]:
field_name = 'apoyo_empleador_entidad_educativa_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency7)
df[field_name].drop_duplicates().reset_index(drop=True)

0    5.0
1    4.0
2    2.0
3    3.0
4    1.0
Name: apoyo_empleador_entidad_educativa_cuarentena, dtype: float64

In [268]:
field_name = 'apoyo_vecinos_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency7)
df[field_name].drop_duplicates().reset_index(drop=True)

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: apoyo_vecinos_cuarentena, dtype: float64

In [269]:
field_name = 'apoyo_alcaldia_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency7)
df[field_name].drop_duplicates().reset_index(drop=True)

0    4.0
1    2.0
2    3.0
3    5.0
4    1.0
Name: apoyo_alcaldia_cuarentena, dtype: float64

In [270]:
field_name = 'apoyo_gobierno_nacional_cuarentena'
convert_colum_categorical_to_numeric(field_name,dict_frecuency7)
df[field_name].drop_duplicates().reset_index(drop=True)

0    4.0
1    2.0
2    3.0
3    1.0
4    5.0
Name: apoyo_gobierno_nacional_cuarentena, dtype: float64

In [271]:
df['sexo'].drop_duplicates().reset_index(drop=True)

0                  Mujer
1                 Hombre
2    Prefiero no decirlo
3             No binario
4                    NaN
Name: sexo, dtype: object

In [272]:
dict_frecuency8 = {
    'Mujer':4,
    'Hombre':3,
    'No binario':2,
    'Prefiero no decirlo':1
}

In [273]:
field_name = 'sexo'
convert_colum_categorical_to_numeric(field_name,dict_frecuency8)
df[field_name].drop_duplicates().reset_index(drop=True)

0    4.0
1    3.0
2    1.0
3    2.0
Name: sexo, dtype: float64

En el caso de la edad usaré en este análisis numérico la variable edad y no rango_edad

In [274]:
df['edad'].dtype

dtype('float64')

In [275]:
field_name = 'diagnostico_enfermedades'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2.0
1    1.0
2    0.0
Name: diagnostico_enfermedades, dtype: float64

In [276]:
df['nivel_educativo'].drop_duplicates().reset_index(drop=True)

0            Título universitario
1              Título de posgrado
2    Título técnico o tecnológico
3             Secundaria completa
4           Secundaria incompleta
5                             NaN
6               Primaria completa
7             Primaria incompleta
Name: nivel_educativo, dtype: object

In [277]:
dict_frecuency9 = {
    'Título de posgrado': 7,
    'Título universitario': 6,
    'Título técnico o tecnológico': 5,
    'Secundaria completa': 4,
    'Secundaria incompleta': 3,
    'Primaria completa': 2,
    'Primaria incompleta': 1,
}

In [278]:
field_name = 'nivel_educativo'
convert_colum_categorical_to_numeric(field_name,dict_frecuency9)
df[field_name].drop_duplicates().reset_index(drop=True)

0    6.0
1    7.0
2    5.0
3    4.0
4    3.0
5    2.0
6    1.0
Name: nivel_educativo, dtype: float64

In [279]:
df['localidad_residencia'].drop_duplicates().reset_index(drop=True)

0                Fontibón
1                    Suba
2     No reside en Bogotá
3                Engativá
4          Ciudad Bolívar
5             Teusaquillo
6               Chapinero
7                Santa Fe
8                 Usaquén
9           Puente Aranda
10         Barrios Unidos
11                Kennedy
12             Tunjuelito
13           Los Mártires
14                   Bosa
15          La Candelaria
16     Rafael Uribe Uribe
17         Antonio Nariño
18          San Cristóbal
19                No sabe
20                   Usme
21                Sumapaz
Name: localidad_residencia, dtype: object

In [280]:
localidades = df['localidad_residencia'].drop_duplicates().reset_index(drop=True).to_list()
localidades.remove('No sabe')
x = 1
dict_frecuency10 = {}
for localidad in localidades:
    dict_frecuency10[localidad] = x
    x += 1
dict_frecuency10['No sabe'] = 0
dict_frecuency10

{'Fontibón': 1,
 'Suba': 2,
 'No reside en Bogotá': 3,
 'Engativá': 4,
 'Ciudad Bolívar': 5,
 'Teusaquillo': 6,
 'Chapinero': 7,
 'Santa Fe': 8,
 'Usaquén': 9,
 'Puente Aranda': 10,
 'Barrios Unidos': 11,
 'Kennedy': 12,
 'Tunjuelito': 13,
 'Los Mártires': 14,
 'Bosa': 15,
 'La Candelaria': 16,
 'Rafael Uribe Uribe': 17,
 'Antonio Nariño': 18,
 'San Cristóbal': 19,
 'Usme': 20,
 'Sumapaz': 21,
 'No sabe': 0}

In [281]:
field_name = 'localidad_residencia'
convert_colum_categorical_to_numeric(field_name,dict_frecuency10)
df[field_name].drop_duplicates().reset_index(drop=True)

0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19     0
20    20
21    21
Name: localidad_residencia, dtype: int64

In [282]:
df['situacion_laboral'].drop_duplicates().reset_index(drop=True)

0                                          Independiente
1                                               Empleado
2                                            Desempleado
3                Vinculación por prestación de servicios
4                                             Pensionado
                             ...                        
120                                         Líder social
121                                  Contratista por ops
122                                              Traders
123                             Prestación de  servicios
124    Contratista con contrato Vigente hasta 21 de J...
Name: situacion_laboral, Length: 125, dtype: object

In [283]:
field_name = 'condiciones_vivienda_tranquilidad_comodidad'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2.0
1    1.0
Name: condiciones_vivienda_tranquilidad_comodidad, dtype: float64

In [284]:
df['num_personas_hogar'].drop_duplicates().reset_index(drop=True)

0             Cuatro
1                Dos
2               Tres
3    Una (vivo solo)
4              Cinco
5         Seis o más
6                NaN
Name: num_personas_hogar, dtype: object

In [285]:
df['num_personas_5_menos'].drop_duplicates().reset_index(drop=True)

0           Una
1       Ninguna
2           Dos
3        Cuatro
4          Tres
5    Seis o más
6         Cinco
Name: num_personas_5_menos, dtype: object

In [286]:
dict_frecuency11 = {
    'Ninguna':0,
    'Una':1,
    'Una (vivo solo)':1,
    'Dos':2,
    'Tres':3,
    'Cuatro':4,
    'Cinco':5,
    'Seis o más':6
}

In [287]:
field_name = 'num_personas_hogar'
convert_colum_categorical_to_numeric(field_name,dict_frecuency11)
df[field_name].drop_duplicates().reset_index(drop=True)

0    4.0
1    2.0
2    3.0
3    1.0
4    5.0
5    6.0
Name: num_personas_hogar, dtype: float64

In [288]:
field_name = 'num_personas_5_menos'
convert_colum_categorical_to_numeric(field_name,dict_frecuency11)
df[field_name].drop_duplicates().reset_index(drop=True)

0    1
1    0
2    2
3    4
4    3
5    6
6    5
Name: num_personas_5_menos, dtype: int64

In [289]:
field_name = 'num_personas_6_18'
convert_colum_categorical_to_numeric(field_name,dict_frecuency11)
df[field_name].drop_duplicates().reset_index(drop=True)

0    1
1    0
2    2
3    3
4    4
5    5
6    6
Name: num_personas_6_18, dtype: int64

In [290]:
field_name = 'num_personas_19_64'
convert_colum_categorical_to_numeric(field_name,dict_frecuency11)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2
1    1
2    3
3    4
4    0
5    5
6    6
Name: num_personas_19_64, dtype: int64

In [291]:
field_name = 'num_personas_65_mas'
convert_colum_categorical_to_numeric(field_name,dict_frecuency11)
df[field_name].drop_duplicates().reset_index(drop=True)

0    0
1    1
2    2
3    4
4    3
5    6
6    5
Name: num_personas_65_mas, dtype: int64

In [292]:
field_name = 'oportunidad_expresar_sentimientos'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    2.0
1    1.0
Name: oportunidad_expresar_sentimientos, dtype: float64

In [293]:
df['frecuencia_actividad_fisica_mes'].drop_duplicates().reset_index(drop=True)

0                     Ninguna vez
1    Menos de una vez a la semana
2      Más de una vez a la semana
3             Una vez a la semana
4                             NaN
Name: frecuencia_actividad_fisica_mes, dtype: object

In [294]:
dict_frecuency12 = {
    'Ninguna vez':0,
    'Una vez a la semana':1,
    'Menos de una vez a la semana':2,    
    'Más de una vez a la semana':3
}

In [295]:
field_name = 'frecuencia_actividad_fisica_mes'
convert_colum_categorical_to_numeric(field_name,dict_frecuency12)
df[field_name].drop_duplicates().reset_index(drop=True)

0    0.0
1    2.0
2    3.0
3    1.0
Name: frecuencia_actividad_fisica_mes, dtype: float64

In [296]:
field_name = 'diagnostico_salud_mental'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0    1.0
1    2.0
2    0.0
Name: diagnostico_salud_mental, dtype: float64

In [297]:
dict_frecuency4['No aplica']=-1

In [298]:
field_name = 'continuidad_tratamiento_salud_mental'
convert_colum_categorical_to_numeric(field_name,dict_frecuency4)
df[field_name].drop_duplicates().reset_index(drop=True)

0   -1.0
1    2.0
2    1.0
3    0.0
Name: continuidad_tratamiento_salud_mental, dtype: float64

In [299]:
df.dtypes

timestamp                                        object
confianza_vecinos                               float64
frecuencia_habla_vecinos_mes_pasado             float64
frecuencia_habla_vecinos_mes_normal             float64
frecuencia_habla_amigos_familia_mes_pasado      float64
frecuencia_habla_amigos_familia_mes_normal      float64
cumplimiento_autocuidado_personal                 int64
cumplimiento_autocuidado_familiares               int64
cumplimiento_autocuidado_vecinos                  int64
estrategia_comunicacion_clara                     int64
estrategia_comunicacion_suficiente                int64
estrategia_comunicacion_veridica                  int64
estrategia_comunicacion_oportuna                  int64
estrategia_comunicacion_contradictoria            int64
medios_info_alcaldia                             object
sabe_actuar_sospechas_contagio                  float64
info_suficiente_COVID                           float64
info_riesgos_COVID_alcaldia                     

In [300]:
df_numerico = df.select_dtypes(include='number')
df_numerico

,confianza_vecinos,frecuencia_habla_vecinos_mes_pasado,frecuencia_habla_vecinos_mes_normal,frecuencia_habla_amigos_familia_mes_pasado,frecuencia_habla_amigos_familia_mes_normal,cumplimiento_autocuidado_personal,cumplimiento_autocuidado_familiares,cumplimiento_autocuidado_vecinos,estrategia_comunicacion_clara,estrategia_comunicacion_suficiente,...,num_personas_hogar,num_personas_5_menos,num_personas_6_18,num_personas_19_64,num_personas_65_mas,oportunidad_expresar_sentimientos,frecuencia_actividad_fisica_mes,exposicion_violencia_hogar,diagnostico_salud_mental,continuidad_tratamiento_salud_mental
0,3.0,1.0,4.0,6.0,5.0,5,5,5,2,2,...,4.0,1,1,2,0,2.0,0.0,1.0,1.0,-1.0
1,2.0,1.0,1.0,6.0,4.0,5,4,3,1,1,...,2.0,0,0,1,0,2.0,2.0,1.0,2.0,2.0
2,2.0,5.0,5.0,6.0,6.0,5,5,3,2,2,...,3.0,0,0,3,0,2.0,3.0,1.0,1.0,-1.0
3,3.0,5.0,5.0,6.0,6.0,5,4,4,2,2,...,4.0,0,0,4,0,2.0,3.0,1.0,2.0,1.0
4,3.0,4.0,4.0,5.0,5.0,4,4,4,2,0,...,1.0,0,0,1,0,2.0,1.0,1.0,1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,1.0,2.0,4.0,6.0,6.0,5,4,1,2,1,...,2.0,0,0,2,0,1.0,0.0,2.0,1.0,-1.0
4994,1.0,1.0,1.0,4.0,5.0,4,3,3,2,1,...,3.0,0,1,2,0,2.0,0.0,1.0,1.0,-1.0
4995,3.0,4.0,3.0,5.0,6.0,3,3,2,0,2,...,5.0,1,2,2,0,1.0,2.0,2.0,1.0,-1.0
4996,2.0,3.0,3.0,5.0,5.0,4,4,2,2,2,...,3.0,0,0,3,0,2.0,1.0,1.0,1.0,-1.0


In [301]:
cur_dir = os.getcwd()
df_numerico.to_csv(os.path.join(cur_dir,'data','df_numeric.csv'),index=False)